<a href="https://colab.research.google.com/github/Notskx1950/NutritionRecognize/blob/main/NutritionAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models

In [2]:
from torch.utils.data import Dataset, DataLoader

In [3]:
from PIL import Image

In [4]:
import kagglehub
# import food dataset
# Download latest version
path = kagglehub.dataset_download("dansbecker/food-101")

print("Path to dataset files:", path)

100%|██████████| 9.38G/9.38G [06:22<00:00, 26.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/dansbecker/food-101/versions/1


In [5]:
data_dir = '/root/.cache/kagglehub/datasets/dansbecker/food-101/versions/1/food-101/food-101'
batch_size = 32
num_epochs = 5
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
classes_txt_path = os.path.join(data_dir, "meta", "classes.txt")
with open(classes_txt_path, "r") as f:
    classes_list = f.read().splitlines()

In [7]:
print(classes_list)

['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheesecake', 'cheese_plate', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons', 'miso_sou

In [8]:
class Label_encoder:
    def __init__(self, labels):
        self.labels = {label: idx for idx, label in enumerate(labels)}
    def get_label(self, idx):
        return list(self.labels.keys())[idx]
    def get_idx(self, label):
        return self.labels.get(label)

In [9]:
classes_list = classes_list[:20] + ['other']

In [10]:
print(len(classes_list))

21


In [11]:
encoder = Label_encoder(classes_list)

In [12]:
for i in range(len(classes_list)):
  print(encoder.get_label(i),encoder.get_idx(encoder.get_label(i)))

apple_pie 0
baby_back_ribs 1
baklava 2
beef_carpaccio 3
beef_tartare 4
beet_salad 5
beignets 6
bibimbap 7
bread_pudding 8
breakfast_burrito 9
bruschetta 10
caesar_salad 11
cannoli 12
caprese_salad 13
carrot_cake 14
ceviche 15
cheesecake 16
cheese_plate 17
chicken_curry 18
chicken_quesadilla 19
other 20


In [13]:
class Food101Dataset(Dataset):
    def __init__(self, root_dir, split="train", transform=None):
        """
        Args:
            root_dir (str): Path to the 'food-101' folder.
            split (str): 'train' or 'test'
            transform: torchvision transforms to apply.
        """
        self.root_dir = root_dir
        # No need to add /
        self.images_dir = os.path.join(self.root_dir, "images")
        self.split = split
        self.transform = transform

        # Read image identifiers from e.g., "meta/train.txt"
        split_file = os.path.join(root_dir, "meta", f"{split}.txt")
        with open(split_file, "r") as f:
            self.samples = f.read().splitlines()


    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        # e.g., "apple_pie/1005641"
        relative_path = self.samples[idx]
        # Split "apple_pie/1005641" into class_name = "apple_pie", image_id = "1005641"
        class_name, image_id = relative_path.split('/')
        img_path = os.path.join(self.images_dir, class_name,image_id + ".jpg")

        # Load the image
        image = Image.open(img_path).convert("RGB")

        # Get the class index
        label = encoder.get_idx(class_name)

        # Apply any specified transforms
        if self.transform:
            image = self.transform(image)

        return image, label


In [14]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [15]:
train_dataset = Food101Dataset(root_dir=data_dir, split="train", transform=train_transforms)
test_dataset  = Food101Dataset(root_dir=data_dir, split="test", transform=test_transforms)

In [16]:
def filter_dataset(dataset):
    """Filters out samples with class names not in the encoder."""
    filtered_samples = []
    for sample in dataset.samples:
        class_name, _ = sample.split('/')
        if encoder.get_idx(class_name) is not None:
            filtered_samples.append(sample)
    dataset.samples = filtered_samples
    return dataset

In [17]:
train_dataset = filter_dataset(train_dataset)
test_dataset = filter_dataset(test_dataset)

In [18]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [19]:
model = models.resnet18(pretrained=True)
num_classes = 21
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 126MB/s]


In [20]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

In [ ]:
for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% || "
          f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%")

# Save your model
torch.save(model.state_dict(), "food101_resnet18.pth")
print("Training finished and model saved.")